In [9]:
# Importar bibliotecas
import json
import pandas as pd

pd.set_option('display.max_colwidth', None)

In [2]:
# Definir o ticker e importar notícias
ticker = 'PETR4'

In [3]:
# Verificar o dataset importado
with open(f'../crawlers/suno/results/suno-{ticker.lower()}.json', encoding='utf8') as json_file:
    data = json.load(json_file)

df = pd.DataFrame(data)
df.sample(5)

,topic,title,date,search_date,url,tags
650,Negócios,"Petrobras (PETR4) contrata 12,9 milhões de met...",08/04/2021 13:03,2022-05-03 16:11:59,https://www.suno.com.br/noticias/petrobras-pet...,"[empresas estatais, gás natural, Jair Bolsonar..."
385,Mercado,Ibovespa abre em alta aos 116 mil pontos com a...,10/09/2021 10:49,2022-05-03 16:06:02,https://www.suno.com.br/noticias/ibovespa-abre...,"[Assaí (ASAI3), B3 (B3SA3), Bolsa de Valores d..."
456,Mercado,"Ibovespa sobe 1,6% junto com NY, após discurso...",27/08/2021 17:57,2022-05-03 16:07:10,https://www.suno.com.br/noticias/ibovespa-alta...,"[americanas S.A (AMER3), Banco Inter (BIDI11/B..."
722,Negócios,Petrobras (PETR4): Gerente de RH suspeito de i...,05/04/2021 08:59,2022-05-03 16:13:29,https://www.suno.com.br/noticias/petrobras-pet...,[Petrobras (PETR4)]
991,Negócios,Petrobras (PETR4): produção na bacia de Campos...,23/10/2020 20:18,2022-05-03 16:19:55,https://www.suno.com.br/noticias/petrobras-pet...,"[PETR3, Petrobras (PETR4)]"


In [4]:
# Selecionar dados de 2020 - Presente

# Convert date to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')

# Set date column as index
df.set_index('date', inplace=True)

# Order by date
df.sort_index(inplace=True)

In [5]:
# Filtrar notícias do tópico 'negócios' para a Suno
# Verificar o similar em MoneyTimes

df = df.loc['2020':,][df['topic'] == 'Negócios']

In [6]:
# Importar o módulo de classificaçao de sentimentos
from sent_classifier import SnorkelSentimentClassifier

sc = SnorkelSentimentClassifier(df)

In [7]:
# Executar o módulo de classificação de sentimentos
df_results = sc.apply_rules(df)

100%|██████████| 500/500 [00:00<00:00, 1687.80epoch/s]


In [11]:
df_results.head(10)

,title_raw,search_date,label_class
date,,,
2020-07-16 08:27:00,Agenda do Dia: JHSF; Tim; Oi; Petrobras; Metalfrio,2022-05-03 16:22:46,NEUTRAL
2020-07-16 09:46:00,Petrobras (PETR4) conclui venda de campos pelo valor total de US$ 372 mi,2022-05-03 16:22:45,POSITIVE
2020-07-16 19:04:00,Petrobras (PETR4): exportação de petróleo para China deve ser reduzida,2022-05-03 16:22:44,NEUTRAL
2020-07-17 09:06:00,Agenda do Dia: Moura Dubeux; Tenda; Petrobras; Hering; Enauta,2022-05-03 16:22:44,NEUTRAL
2020-07-18 13:00:00,Petrobras (PETR4) eleva em 4% preço da gasolina e 6% do diesel nas refinarias,2022-05-03 16:22:43,NEUTRAL
2020-07-20 21:13:00,Engie Brasil (EGIE3) assina contrato para compra de 10% da TAG,2022-05-03 16:22:45,NEUTRAL
2020-07-21 08:39:00,Agenda do Dia: Petrobras; Marfrig; Oi; Vale; Ser Educacional; Via Varejo,2022-05-03 16:22:45,NEUTRAL
2020-07-21 09:06:00,Petrobras (PETR4) confirma venda de 10% da TAG,2022-05-03 16:22:46,NEUTRAL
2020-07-21 10:14:00,Petrobras tem decisão favorável da Justiça em processo de PIS e Cofins pagos a mais,2022-05-03 16:22:46,NEUTRAL
